# web dApp

## Build the Frontend

Let's build the shell in which we will populate data. In Flask (our backend uses Python) we can use the `${{ variable }}` or `${{ logic variable}}` as this will be rendered out when a user loads the page.

The download button routes to `/download`, which will trigger more backend logic and then route back to the index to populate our new data.

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>web3 dApp</title>
</head>
<body>
    <div class="container">
        <!-- Box for Ethereum Price -->
        <div class="box">
            <h1>Ethereum (ETH) Price</h1>
            <p>The current price of Ethereum (ETH) is: ${{ eth_price }}</p>
        </div>

        <!-- Box for Image Download -->
        <div class="box">
            {% if image_available %}
                <img src="{{ url_for('static', filename=file_key) }}" alt="Downloaded Image">
            {% else %}
                <p>Image not available. Click download to retrieve the image.</p>
            {% endif %}
            <a href="/download" class="btn">Download File</a>
        </div>
    </div>
</body>
</html>
```

### Add some style

```html
<style>
        /* Add styles here */
        body {
            font-family: 'Arial', sans-serif;
            background-color: #121212;
            color: #ffffff;
            margin: 0;
            padding: 20px;
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
        }
        .container {
            display: flex;
            gap: 20px;
        }
        .box {
            flex: 1;
            padding: 20px;
            border: 1px solid #333;
            border-radius: 8px;
            background-color: #1e1e1e;
        }
        h1 {
            font-size: 24px;
            margin-bottom: 20px;
        }
        p {
            font-size: 18px;
            line-height: 1.6;
        }
        img {
            max-width: 100%;
            border-radius: 8px;
            margin-top: 20px;
        }
        .btn {
            display: inline-block;
            padding: 10px 20px;
            margin-top: 20px;
            background-color: #af594c;
            color: #ffffff;
            text-decoration: none;
            border-radius: 5px;
            transition: background-color 0.3s ease;
        }
        .btn:hover {
            background-color: #a04351;
        }
    </style>
```

## Build the web dApp backend

### Imports

```python
# Using Flask as our web framework for Python
from flask import Flask, render_template, send_file, redirect, url_for

# web3 is our library for dealing with calls to the blockchain
from web3 import Web3

# boto3 is a standard library for dealing with S3
import boto3
from botocore.exceptions import NoCredentialsError


from os.path import exists
```

### Build the Routes

```python
app = Flask(__name__)

# Main route when they visit the index
@app.route('/')

# Route triggered on download button press
@app.route('/download')

# Run the Flask app loop
if __name__ == '__main__':
    app.run(debug=True)
```

### Write function that gets Ethereum price

```python
def get_eth_price():
    # Change this to use your own RPC URL
    # web3 = Web3(Web3.HTTPProvider('https://rpc.ankr.com/eth_goerli'))
    web3 = Web3(Web3.HTTPProvider('https://eth-goerli.gateway.pokt.network/v1/lb/961ce20a'))
    # AggregatorV3Interface ABI
    abi = '[{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"description","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint80","name":"_roundId","type":"uint80"}],"name":"getRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]'
    # Price Feed address
    addr = '0xD4a33860578De61DBAbDc8BFdb98FD742fA7028e'

    # Set up contract instance
    contract = web3.eth.contract(address=addr, abi=abi)
    # Make call to latestRoundData()
    latestData = contract.functions.latestRoundData().call()

    eth_price = latestData[1] / (10 ** 8)

    # Format to 2 decimal places and convert to string
    formatted_price = "{:,.2f}".format(eth_price)
    
    return formatted_price
```

### Add Eth Price function to index route

Since we haven't written the logic for getting the image (that's next), our index page will render that the image is not yet available (what we want).

```python
@app.route('/')
def index():
    eth_price = get_eth_price()
    file_key = "poap.png"
    image_available = exists('static/' + file_key)
    return render_template('index.html', eth_price=eth_price, file_key=file_key, image_available=image_available)
```

### Write logic to download image from Storj using S3 API

We will pass the `access_key`, `secret_key`, `endpoint_url`, `file_key`, and `destination_route` in the `/download` route. That keeps this function generic so we could have multiple different download routes to download different objects. 

```python
def download_from_storj_s3(bucket_name, file_key, destination_path, access_key, secret_key, endpoint_url):
    # Initialize a session using boto3
    session = boto3.session.Session()
    s3_client = session.client(
        service_name='s3',
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key,
        endpoint_url=endpoint_url,
    )

    try:
        s3_client.download_file(bucket_name, file_key, destination_path)
        print(f"File '{file_key}' downloaded successfully to '{destination_path}'.")
    except NoCredentialsError:
        print("Credentials not available")
```

### Write the download route

When the user clicks the download button we made on the frontend, it will trigger this route and pass all the variables we need to our `download_from_storj_s3(...)` function. Notice how the `return` will pass us right back to the index, without reloading the page or aything, it will just pass back the new data for rendering. 

```python
@app.route('/download')
def download_file():
    bucket_name = "demo-bucket"
    file_key = "web3 Workshop Poap.png"
    destination_path = "static/poap.png"
    access_key = "jwxwkd3pirkb77ngtiwxr4ajphoq"
    secret_key = "j3omtog5wlcyjudtdzq3krxchf5floizwzjinxljtjw77wouuhlpq"
    endpoint_url = "https://gateway.storjshare.io"

    download_from_storj_s3(bucket_name, file_key, destination_path, access_key, secret_key, endpoint_url)
    
    return redirect(url_for('index'))
```

## Running the app

To spin up the Flask server, all you need to do is from a terminal in the directory where the project is stored, run `python app.py` and the Flask server will run the code and you can navigate to it on your localhost. 

The goal of this simple example is to show you how you can integrate web3 technologies such as decentrlaized apis, data, and storage into web2 friendly development workflows.

This site is also provided as static as `static_site.html` where the server-side processing done in Python is replaced by JavaScript that runs client-side in the browser. This means we can deploy it to IPFS and the functionality remains the same, making an unstoppable web dApp!